Prerequsites: 
pip install opendatasets
pip install pandas 

In [13]:
pd.reset_option('display.max_rows')

# Reset the 'display.max_columns' option to its default value (20 by default)
pd.reset_option('display.max_columns')

In [14]:
import opendatasets as od
import pandas as pd
import requests
import csv
from statistics import mode, median

## Dataset 1: Property Price Register Ireland
Downloading Data from Kaggle.com and store in a dataframe to be cleaned.

In [15]:
od.download("https://www.kaggle.com/datasets/erinkhoo/property-price-register-ireland/data") #download dataset from kaggle

propertyPrice_df = pd.read_csv('property-price-register-ireland/Property_Price_Register_Ireland-28-05-2021.csv') #store dataset in a data frame. 
propertyPrice_df #display raw data frame

Skipping, found downloaded files in "./property-price-register-ireland" (use force=True to force download)


,SALE_DATE,ADDRESS,POSTAL_CODE,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,PROPERTY_SIZE_DESC
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,343000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
1,2010-01-03,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,0,1,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,438500.0,0,0,Second-Hand Dwelling house /Apartment,NaN
3,2010-01-04,"1 The Haven, Mornington",NaN,Meath,400000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
4,2010-01-04,"11 Melville Heights, Kilkenny",NaN,Kilkenny,160000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
...,...,...,...,...,...,...,...,...,...
476740,2021-05-28,"GRANGE STORES, GRANGE, MULLINGAR",NaN,Westmeath,150000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
476741,2021-05-28,"MAIN STREET, LOWER BORRIS, BORRIS",NaN,Carlow,170000.0,1,0,Second-Hand Dwelling house /Apartment,NaN
476742,2021-05-28,"MULTYFARNHAM, WESTMEATH",NaN,Westmeath,175000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
476743,2021-05-28,"NO 2 RIVERLODGE, SHRONE, GLENGARIFF",NaN,Cork,240000.0,0,0,Second-Hand Dwelling house /Apartment,NaN


Begin cleaning data set

In [3]:
propertyPrice_df.drop(columns=["ADDRESS", "POSTAL_CODE", "IF_MARKET_PRICE", "PROPERTY_SIZE_DESC" ], inplace=True) #drop columns and store in current data frame
propertyPrice_df #display data frame with removed columns 

,SALE_DATE,COUNTY,SALE_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC
0,2010-01-01,Dublin,343000.0,0,Second-Hand Dwelling house /Apartment
1,2010-01-03,Laois,185000.0,1,New Dwelling house /Apartment
2,2010-01-04,Dublin,438500.0,0,Second-Hand Dwelling house /Apartment
3,2010-01-04,Meath,400000.0,0,Second-Hand Dwelling house /Apartment
4,2010-01-04,Kilkenny,160000.0,0,Second-Hand Dwelling house /Apartment
...,...,...,...,...,...
476740,2021-05-28,Westmeath,150000.0,0,Second-Hand Dwelling house /Apartment
476741,2021-05-28,Carlow,170000.0,0,Second-Hand Dwelling house /Apartment
476742,2021-05-28,Westmeath,175000.0,0,Second-Hand Dwelling house /Apartment
476743,2021-05-28,Cork,240000.0,0,Second-Hand Dwelling house /Apartment


In [4]:
propertyPrice_df = propertyPrice_df.dropna() #drop any rows that contain null values

In [5]:
propertyPrice_df.rename(columns = {
    'SALE_DATE':'SALE DATE',
    'SALE_PRICE':'SALE PRICE',
    'IF_VAT_EXCLUDED':'VAT EXCLUDED',
    'PROPERTY_DESC': 'NEW BUILD' 
}, inplace = True) #rename columns and store in a new dataframe 

In this data set, there are new and old houses.
VAT of 13.5% is applied to houses where VAT is excluded from the sales prices to standardise prices 

In [6]:
newPropertyPrice_df = propertyPrice_df.copy(deep=True) #create a copy to avoid modifying the original 
newPropertyPrice_df.loc[propertyPrice_df['VAT EXCLUDED'] == 1, 'SALE PRICE'] *= 1.135

newPropertyPrice_df.drop(columns=["VAT EXCLUDED"], inplace=True) #drop column and store in current data frame
newPropertyPrice_df

,SALE DATE,COUNTY,SALE PRICE,NEW BUILD
0,2010-01-01,Dublin,343000.0,Second-Hand Dwelling house /Apartment
1,2010-01-03,Laois,209975.0,New Dwelling house /Apartment
2,2010-01-04,Dublin,438500.0,Second-Hand Dwelling house /Apartment
3,2010-01-04,Meath,400000.0,Second-Hand Dwelling house /Apartment
4,2010-01-04,Kilkenny,160000.0,Second-Hand Dwelling house /Apartment
...,...,...,...,...
476740,2021-05-28,Westmeath,150000.0,Second-Hand Dwelling house /Apartment
476741,2021-05-28,Carlow,170000.0,Second-Hand Dwelling house /Apartment
476742,2021-05-28,Westmeath,175000.0,Second-Hand Dwelling house /Apartment
476743,2021-05-28,Cork,240000.0,Second-Hand Dwelling house /Apartment


In [7]:
newPropertyPrice_df.loc[newPropertyPrice_df['NEW BUILD'] == 'New Dwelling house /Apartment', 'NEW BUILD'] = '1'
newPropertyPrice_df.loc[newPropertyPrice_df['NEW BUILD'] == 'Second-Hand Dwelling house /Apartment', 'NEW BUILD'] = '0'
newPropertyPrice_df

,SALE DATE,COUNTY,SALE PRICE,NEW BUILD
0,2010-01-01,Dublin,343000.0,0
1,2010-01-03,Laois,209975.0,1
2,2010-01-04,Dublin,438500.0,0
3,2010-01-04,Meath,400000.0,0
4,2010-01-04,Kilkenny,160000.0,0
...,...,...,...,...
476740,2021-05-28,Westmeath,150000.0,0
476741,2021-05-28,Carlow,170000.0,0
476742,2021-05-28,Westmeath,175000.0,0
476743,2021-05-28,Cork,240000.0,0


## Dataset 2: Ireland's Population 
Download dataset from www.cso.ie 

In [8]:
population_df = pd.read_csv('./census-population/F3001.20231026T151057.csv') #store dataset in a data frame. 
population_df #display raw data frame

,Statistic Label,Census Year,County and City,Detailed Marital Status,Sex,UNIT,VALUE
0,Population,2011,State,All marital status,Both sexes,Number,4588252
1,Population,2011,State,All marital status,Male,Number,2272699
2,Population,2011,State,All marital status,Female,Number,2315553
3,Population,2011,State,Single,Both sexes,Number,2484625
4,Population,2011,State,Single,Male,Number,1285612
...,...,...,...,...,...,...,...
2227,Population,2022,Monaghan,Divorced,Male,Number,646
2228,Population,2022,Monaghan,Divorced,Female,Number,975
2229,Population,2022,Monaghan,Widowed,Both sexes,Number,2921
2230,Population,2022,Monaghan,Widowed,Male,Number,763


Begin cleaning data set

In [9]:
population_df.drop(columns=['UNIT'], inplace=True) #drop columns and store in current data frame
population_df #display data frame with removed columns 

,Statistic Label,Census Year,County and City,Detailed Marital Status,Sex,VALUE
0,Population,2011,State,All marital status,Both sexes,4588252
1,Population,2011,State,All marital status,Male,2272699
2,Population,2011,State,All marital status,Female,2315553
3,Population,2011,State,Single,Both sexes,2484625
4,Population,2011,State,Single,Male,1285612
...,...,...,...,...,...,...
2227,Population,2022,Monaghan,Divorced,Male,646
2228,Population,2022,Monaghan,Divorced,Female,975
2229,Population,2022,Monaghan,Widowed,Both sexes,2921
2230,Population,2022,Monaghan,Widowed,Male,763


In [10]:
population_df = population_df.dropna() #drop any rows that contain null values
rows_dropped = len(population_df) - len(population_df.dropna())
print(f"Number of rows dropped: {rows_dropped}")

Number of rows dropped: 0


In [11]:
population_df = population_df[population_df['Sex'] == 'Both sexes']
population_df = population_df[population_df['Detailed Marital Status'] == 'All marital status']
population_df = population_df[population_df['County and City'] != 'State']
population_df.drop(columns=['Detailed Marital Status','Sex'], inplace=True) #drop columns and store in current data frame
population_df

,Statistic Label,Census Year,County and City,VALUE
24,Population,2011,Carlow,54612
48,Population,2011,Dublin City,527612
72,Population,2011,Dún Laoghaire-Rathdown,206261
96,Population,2011,Fingal,273991
120,Population,2011,South Dublin,265205
...,...,...,...,...
2112,Population,2022,Roscommon,70259
2136,Population,2022,Sligo,70198
2160,Population,2022,Cavan,81704
2184,Population,2022,Donegal,167084


In [3]:
population_df.rename(columns={'County and City': 'County'}, inplace = True) #rename columns and store in a new dataframe 


uniqueCounties = population_df['County'].unique()
print(uniqueCounties)

population_df['County'] = population_df['County'].replace(
    {
        ' City': '',
        ' County' : '',
        ' City and County' : '',
        'South Dublin': 'Dublin',
        'Dún Laoghaire-Rathdown': 'Dublin',
        'Fingal': 'Dublin',
        'Cork City and Cork County':'Cork',
    }, regex=True)


population_df

NameError: name 'population_df' is not defined

In [ ]:
result_df = population_df.groupby(['Census Year', 'County'])['VALUE'].sum().reset_index()
print(result_df.head(84))

##  ---


Calculating Mean, Mode and median for the whole data set

In [ ]:
mean = df["SALE_PRICE"].mean()
mode_value = mode(df["SALE_PRICE"])
median_value = median(df["SALE_PRICE"])

print("Mean SALE_PRICE:", mean)
print("Mode SALE_PRICE:", mode_value)
print("Median SALE_PRICE:", median_value)

In [ ]:
# Assuming you have a DataFrame 'df' with columns 'COUNTY' and 'SALE_PRICE'

# Calculate the mean per county
mean_per_county = df.groupby('COUNTY')['SALE_PRICE'].mean()

# Calculate the mode per county
mode_per_county = df.groupby('COUNTY')['SALE_PRICE'].apply(lambda x: mode(x) if len(x) > 0 else None)

# Calculate the median per county
median_per_county = df.groupby('COUNTY')['SALE_PRICE'].median()

# Create a new DataFrame with the results
summary_df = pd.DataFrame({'Mean_SALE_PRICE': mean_per_county, 'Mode_SALE_PRICE': mode_per_county, 'Median_SALE_PRICE': median_per_county})

# Print or use summary_df as needed
print(summary_df)

In [ ]:
# Extract the year from the 'SALE_DATE' column
df['SALE_YEAR'] = pd.to_datetime(df['SALE_DATE']).dt.year

# Group by 'COUNTY' and 'SALE_YEAR'
grouped = df.groupby(['COUNTY', 'SALE_YEAR'])

# Calculate the mean, mode, and median per county and year
mean_per_county_year = grouped['SALE_PRICE'].mean()
mode_per_county_year = grouped['SALE_PRICE'].apply(lambda x: mode(x) if len(x) > 0 else None)
median_per_county_year = grouped['SALE_PRICE'].median()

# Create a new DataFrame with the results
summary_df = pd.DataFrame({'Mean_SALE_PRICE': mean_per_county_year, 'Mode_SALE_PRICE': mode_per_county_year, 'Median_SALE_PRICE': median_per_county_year}).reset_index()

# Print or use summary_df as needed
print(summary_df)

In [ ]:
display(summary_df.to_string())
df.to_csv(r'property-price-register-ireland/export_dataframe.csv', index=False, header=True)

print(df)

In [ ]:
# Count the number of rows for each value in IF_MARKET_PRICE
#market_price_counts = df['IF_MARKET_PRICE'].value_counts()

# Count the number of rows for each value in IF_VAT_EXCLUDED
vat_excluded_counts = df['IF_VAT_EXCLUDED'].value_counts()

#print("Count of each value in IF_MARKET_PRICE:")
#print(market_price_counts)

print("\nCount of each value in IF_VAT_EXCLUDED:")
print(vat_excluded_counts)

Population data from https://data.cso.ie